## Fake News Detection Using NLP

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/FakeNewsDataset/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Drop Nan Values

In [5]:
df=df.dropna()

Get the Independent Features

In [6]:
X=df.drop('label',axis=1)

 Get the Dependent features

In [7]:
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

#Data Pre-processing

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
tf.__version__

'2.3.0'

Vocabulary size

In [13]:
voc_size=5000

###Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus[3]

'civilian kill singl us airstrik identifi'

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:10]

[[1789, 130, 3735, 1761, 2306, 1774, 1700, 2229, 478, 4685],
 [3162, 2659, 1413, 4940, 616, 2419, 2952],
 [2976, 935, 688, 3773],
 [781, 4413, 905, 1156, 4023, 3007],
 [4740, 616, 3656, 4252, 1616, 3924, 616, 750, 2008, 2864],
 [2701,
  65,
  1794,
  4465,
  504,
  997,
  4123,
  4666,
  1690,
  2467,
  3739,
  3974,
  1685,
  1141,
  2952],
 [1363, 1415, 3609, 3894, 4801, 2718, 1810, 451, 4309, 2189, 1352],
 [2169, 338, 4811, 2822, 4925, 2126, 997, 805, 4309, 2189, 1352],
 [667, 1717, 4360, 2028, 4788, 2196, 2060, 1141, 997, 999],
 [2676, 4105, 2964, 1107, 3911, 1909, 809, 121]]

## Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2229  478 4685]
 [   0    0    0 ...  616 2419 2952]
 [   0    0    0 ...  935  688 3773]
 ...
 [   0    0    0 ... 4309 2189 1352]
 [   0    0    0 ... 2957 3767  739]
 [   0    0    0 ... 4065 3930 2863]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1789,
        130, 3735, 1761, 2306, 1774, 1700, 2229,  478, 4685], dtype=int32)

# Model

In [24]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [29]:
from tensorflow.keras.layers import Dropout

In [30]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Model Training

In [31]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
192/192 [==============================] - 7s 37ms/step - loss: 0.3406 - accuracy: 0.8353 - val_loss: 0.2078 - val_accuracy: 0.9105
Epoch 2/50
192/192 [==============================] - 7s 35ms/step - loss: 0.1502 - accuracy: 0.9423 - val_loss: 0.1962 - val_accuracy: 0.9162
Epoch 3/50
192/192 [==============================] - 7s 35ms/step - loss: 0.1100 - accuracy: 0.9579 - val_loss: 0.2148 - val_accuracy: 0.9170
Epoch 4/50
192/192 [==============================] - 7s 34ms/step - loss: 0.0824 - accuracy: 0.9699 - val_loss: 0.2334 - val_accuracy: 0.9150
Epoch 5/50
192/192 [==============================] - 6s 34ms/step - loss: 0.0672 - accuracy: 0.9762 - val_loss: 0.2962 - val_accuracy: 0.9118
Epoch 6/50
192/192 [==============================] - 6s 34ms/step - loss: 0.0535 - accuracy: 0.9820 - val_loss: 0.2855 - val_accuracy: 0.9109
Epoch 7/50
192/192 [==============================] - 6s 34ms/step - loss: 0.0400 - accuracy: 0.9869 - val_loss: 0.3578 - val_accuracy: 0.9089

### Performance Metrics And Accuracy

In [32]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [33]:
y_pred[:10]

array([[1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test,y_pred)

array([[3147,  272],
       [ 283, 2333]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.908036454018227